In [ ]:
# get top papsers; used for GPT calling.

file_path = '/home/jx4237/projects/web_page/cm4ai_tkg_oct/work/data/author_dataset_filtered.csv'
import pandas as pd
meta = pd.read_csv(file_path)

In [3]:
Papers_2m = pd.read_csv('/data/jx4237data/TKG/TKG_JCDL/Bridge2AI_2m/Papers_2m.csv.gz')

In [6]:
paper_author = pd.read_csv('/data/jx4237data/TKG/TKG_JCDL/Bridge2AI_2m/paper_author40k.csv.gz')

In [16]:
# Pre-merge the data and sort
paper_author_merged = paper_author.merge(Papers_2m[['PMID', 'PubYear', 'citation_count']], on=['PMID', 'PubYear'], how='inner')

# Separate the papers from 2017 onwards for citation sorting
paper_author_since_2017 = paper_author_merged[paper_author_merged['PubYear'] >= 2017]

# Sort the DataFrames once by necessary criteria
recent_sorted = paper_author_merged.sort_values(by=['AID', 'PubYear', 'PMID'], ascending=[True, False, True])
cited_sorted = paper_author_since_2017.sort_values(by=['AID', 'citation_count', 'PubYear', 'PMID'], ascending=[True, False, False, True])

# Get the top 5 recent and 5 most cited papers for each author
top_recent = recent_sorted.groupby('AID').head(5)
top_cited = cited_sorted.groupby('AID').head(5)

# Combine the results and remove duplicates
combined_papers = pd.concat([top_recent, top_cited]).drop_duplicates(subset=['AID', 'PMID'])

# Aggregate PMIDs into a comma-separated string for each author
pmids_strings = combined_papers.groupby('AID')['PMID'].apply(lambda pmids: ','.join(map(str, pmids)))

# Merge the pmids_string back to the meta DataFrame
meta = meta.merge(pmids_strings.rename('pmids_string'), left_on='id', right_index=True, how='left')

In [18]:
meta.to_csv('/home/jx4237/projects/web_page/cm4ai_tkg_oct/work/data/author_dataset_filtered.csv', index=False)